# Library for neural networks

Sequential neural networks for supervised learning

#### Imports

In [ ]:
from classes import Neuron, Layer, Network_Model
import helpers as hlp
import pandas as pd

#### Instanciate a model

`add_layer` adds a new layer to the model. The amount of neurons and the desired activation function can be set. Layer 0 is automatically set to be the input layer. Weights and biases are randomly assigned.

In [ ]:
mdl = Network_Model()
mdl.add_layer(3)
mdl.add_layer(16, activation='sigmoid')
mdl.add_layer(11, activation='sigmoid')

Using `plot_network` or `plot_network_pretty`, the Model can be visualized.

In [ ]:
mdl.plot_network_pretty()

In [ ]:
mdl.layers[1].weights

Using the `predict()` method, a prediction is made based on a given input-vector. Since the model is not trained yet, the output is basically random.

In [ ]:
input = [255, 255, 255]
mdl.predict(input)

In [ ]:
mdl.plot_network_pretty()

In [ ]:
df_orig = pd.read_excel('datasets\\rgb.xlsx')
df = df_orig.copy()
df

In [ ]:
df_clean = df.drop(columns=['Name'])
data = df_clean.values.tolist()
test = data[:5]

In [ ]:
mdl.train(test)